In [4]:
import json

# Load the local Figma design JSON file
with open("figma_design_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Flatten the node tree
all_nodes = {}

def flatten_nodes(node):
    if isinstance(node, dict):
        if 'id' in node:
            all_nodes[node['id']] = node
        if 'children' in node:
            for child in node['children']:
                flatten_nodes(child)

flatten_nodes(data['document'])

# Extract interactions using the "interactions" field
interactions_json = []

for node_id, node in all_nodes.items():
    if 'interactions' in node and isinstance(node['interactions'], list):
        for interaction in node['interactions']:
            trigger = interaction.get("trigger", {}).get("type", "UNKNOWN")
            actions = interaction.get("actions", [])

            for action in actions:
                if not action:
                    continue
                action_type = action.get("type", "UNKNOWN")
                destination_id = action.get("destinationId")
                destination_name = all_nodes.get(destination_id, {}).get("name", "Unknown") if destination_id else None

                interactions_json.append({
                    "node_id": node_id,
                    "node_name": node.get("name", "(Unnamed)"),
                    "trigger": trigger,
                    "action_type": action_type,
                    "destination_id": destination_id,
                    "destination_name": destination_name
                })
for interaction in interactions_json:
    print(f"🔸 Node Name     : {interaction['node_name']} [{interaction['node_id']}]")
    print(f"   Trigger       : {interaction['trigger']}")
    print(f"   Action Type   : {interaction['action_type']}")
    if interaction["destination_id"]:
        print(f"   Destination   : {interaction['destination_name']} [{interaction['destination_id']}]")
    else:
        print(f"   Destination   : None")
    print("-" * 50)

# Save the interactions to a JSON file
with open("extracted_interactions_from_design.json", "w", encoding="utf-8") as f:
    json.dump(interactions_json, f, indent=4)

print("✅ Interactions extracted and saved to 'extracted_interactions_from_design.json'")


🔸 Node Name     : splash [3:3]
   Trigger       : AFTER_TIMEOUT
   Action Type   : NODE
   Destination   : home [3:4]
--------------------------------------------------
🔸 Node Name     : btn-menu [4:45]
   Trigger       : ON_CLICK
   Action Type   : NODE
   Destination   : menu [3:7]
--------------------------------------------------
🔸 Node Name     : btn-menu [4:45]
   Trigger       : DRAG
   Action Type   : NODE
   Destination   : menu [3:7]
--------------------------------------------------
🔸 Node Name     : Rectangle [3:42]
   Trigger       : ON_CLICK
   Action Type   : NODE
   Destination   : recipe-details [3:5]
--------------------------------------------------
🔸 Node Name     : Prime Rib Roast [4:278]
   Trigger       : ON_CLICK
   Action Type   : NODE
   Destination   : recipe-details [3:5]
--------------------------------------------------
🔸 Node Name     : Rectangle [3:67]
   Trigger       : DRAG
   Action Type   : BACK
   Destination   : None
-------------------------------